In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import typing
import time
from dotenv import load_dotenv
from influxdb import DataFrameClient
import os

In [2]:
load_dotenv()

True

In [ ]:
def doGet(url: str) -> BeautifulSoup:
    return BeautifulSoup(requests.get(url).content, 'html.parser')

In [ ]:
# get all counties of Romania
counties_page = doGet('https://en.wikipedia.org/wiki/Counties_of_Romania')
counties_table_rows = counties_page.select_one('#mw-content-text > div.mw-parser-output > table.wikitable').find('tbody').find_all('tr')[1:]
counties_data = [(elem.text.strip() for elem in row.select('th, td')[:-2]) for row in counties_table_rows]
counties_df = pd.DataFrame(counties_data)
counties_df.rename(columns={0: 'County', 1: 'County Seat', 3: 'Development Region', 4: 'ISO Code', 7: 'NUTS Code', 8: 'Population'}, inplace=True)
counties_df['County Seat'] = [re.sub('\[.*\]', '', value) for value in counties_df['County Seat']]
counties_df.drop(columns=[2, 5, 6], inplace=True)
# easier
counties_df.loc[9]['County Seat'] = 'Bucharest'
counties_df['Population'] = [value.replace(',','') for value in counties_df['Population']]
counties_df['Population'] = pd.to_numeric(counties_df['Population'])
counties_df['County'].loc[9] = 'București'

In [ ]:
counties_df.to_json('../data/counties.json', orient='records')

In [45]:
g16 = pd.read_json('../data/gas_tmp-16.json', orient='records')
g17 = pd.read_json('../data/gas_tmp-17.json', orient='records')
g18 = pd.read_json('../data/gas_tmp-18.json', orient='records')
counties = pd.read_json('../data/counties.json', orient='records')

In [3]:
counties = pd.read_json('../data/counties.json', orient='records')
g19 = pd.read_json('../data/gas_tmp-19.json', orient='records')

In [4]:
counties['ISO Code'] = [f'RO-{value}' for value in counties['ISO Code']]

In [48]:
counties.to_json('../data/counties.json', orient='records')

In [17]:
g16 = g16.merge(counties[['County', 'ISO Code']], how='left', left_on='County', right_on='County')
g17 = g17.merge(counties[['County', 'ISO Code']], how='left', left_on='County', right_on='County')
g18 = g18.merge(counties[['County', 'ISO Code']], how='left', left_on='County', right_on='County')

In [4]:
g19 = g19.merge(counties[['County', 'ISO Code']], how='left', left_on='County', right_on='County')

In [32]:
g16.rename(columns={'ISO Code': 'ISO'}, inplace=True)
g17.rename(columns={'ISO Code': 'ISO'}, inplace=True)
g18.rename(columns={'ISO Code': 'ISO'}, inplace=True)

In [33]:
g16.to_json('../data/gas_tmp-16.json', orient='records')
g17.to_json('../data/gas_tmp-17.json', orient='records')
g18.to_json('../data/gas_tmp-18.json', orient='records')

In [7]:
g19.rename(columns={'ISO Code': 'ISO'}, inplace=True)

In [8]:
g19.to_json('../data/gas_tmp-19.json', orient='records')